## Начало

In [108]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt

pd.set_option('max_colwidth', 400)
pd.set_option('display.width', 500)

df = pd.read_csv('1mcorpus/corpus.en_ru.1m.en', encoding='utf-8', on_bad_lines='skip', delimiter='/n', header=None, index_col=False).tail(50000)

df

C:\Users\1\AppData\Local\Temp\ipykernel_29656\2311108680.py:27: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('1mcorpus/corpus.en_ru.1m.en', encoding='utf-8', on_bad_lines='skip', delimiter='/n', header=None, index_col=False).tail(50000)
C:\Users\1\AppData\Local\Temp\ipykernel_29656\2311108680.py:27: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv('1mcorpus/corpus.en_ru.1m.en', encoding='utf-8', on_bad_lines='skip', delimiter='/n', header=None, index_col=False).tail(50000)


,0
950000,"Ensuring you a high quality of service and amenities, Napoleon Hostel has 3 stars."
950001,"One will be able to meet the Old Slavic god of marriage, Kolodiy, and each son-in-law can ride his beloved mother-in-law in the wagon or sleigh, to feed her with dumplings and pancakes."
950002,"Located 250 metres from the centre of Žabljak, Apartments Vuković is set in the Nature Park Durmitor and surrounded by untouched nature and woods."
950003,We also offer bog trips in ARGO amphibious vehicles.
950004,Lübeck’s Main Station is a direct bus ride or a 15-minute walk away.
...,...
999995,Such actions are called interventions.
999996,"Kaliningrad, May 20, 2013"
999997,"Urinary tract infections in pregnancy A.I. Dyadyk, A.E. Bagriy, N.F. Yarovaya, M.V. Homenko, D.B. Reznikov, Yu.V. Roschin, I.G. Bagriy Urinary tract infections (UTI) associated with pregnancy represents the important problem of obstetrics, urology and nephrology, caused by changes in clinical features UTI, approaches for assessment and treatment UTI and high risk of obstetrical, urologic and n..."
999998,"The default is 8 days for Exchange Server 5.5 and 40 days for Exchange 2000 Server, for Exchange Server 2003, for Exchange Server 2007, and for Exchange Server 2010."


In [109]:
df.to_csv('en.csv')

In [110]:
df2 = pd.read_csv('1mcorpus/corpus.en_ru.1m.ru', encoding='utf-8', on_bad_lines='skip', delimiter='/n', header=None, index_col=False).tail(50000)

df2.to_csv('ru.csv')

C:\Users\1\AppData\Local\Temp\ipykernel_29656\698285661.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv('1mcorpus/corpus.en_ru.1m.ru', encoding='utf-8', on_bad_lines='skip', delimiter='/n', header=None, index_col=False).tail(50000)
C:\Users\1\AppData\Local\Temp\ipykernel_29656\698285661.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df2 = pd.read_csv('1mcorpus/corpus.en_ru.1m.ru', encoding='utf-8', on_bad_lines='skip', delimiter='/n', header=None, index_col=False).tail(50000)


In [111]:
df2.to_csv('ru.csv')

In [112]:
df

,0
950000,"Ensuring you a high quality of service and amenities, Napoleon Hostel has 3 stars."
950001,"One will be able to meet the Old Slavic god of marriage, Kolodiy, and each son-in-law can ride his beloved mother-in-law in the wagon or sleigh, to feed her with dumplings and pancakes."
950002,"Located 250 metres from the centre of Žabljak, Apartments Vuković is set in the Nature Park Durmitor and surrounded by untouched nature and woods."
950003,We also offer bog trips in ARGO amphibious vehicles.
950004,Lübeck’s Main Station is a direct bus ride or a 15-minute walk away.
...,...
999995,Such actions are called interventions.
999996,"Kaliningrad, May 20, 2013"
999997,"Urinary tract infections in pregnancy A.I. Dyadyk, A.E. Bagriy, N.F. Yarovaya, M.V. Homenko, D.B. Reznikov, Yu.V. Roschin, I.G. Bagriy Urinary tract infections (UTI) associated with pregnancy represents the important problem of obstetrics, urology and nephrology, caused by changes in clinical features UTI, approaches for assessment and treatment UTI and high risk of obstetrical, urologic and n..."
999998,"The default is 8 days for Exchange Server 5.5 and 40 days for Exchange 2000 Server, for Exchange Server 2003, for Exchange Server 2007, and for Exchange Server 2010."


In [113]:
df = pd.concat([df, df2],axis=1, ignore_index=True)
df

,0,1
950000,"Ensuring you a high quality of service and amenities, Napoleon Hostel has 3 stars.","Napoleon Hostel, имеющий 3 звезды, обеспечивает высокий уровень обслуживания и удобств."
950001,"One will be able to meet the Old Slavic god of marriage, Kolodiy, and each son-in-law can ride his beloved mother-in-law in the wagon or sleigh, to feed her with dumplings and pancakes.","Также народ сможет встретиться со старославянским богом брака Колодием, а каждый зять сможет покатать любимую тещу на телеге или санях, покормить ее варениками и блинчиками."
950002,"Located 250 metres from the centre of Žabljak, Apartments Vuković is set in the Nature Park Durmitor and surrounded by untouched nature and woods.","Отель Enigma расположен в самом центре национального парка Дурмитор, в 1,2 км от центра города Жабляк."
950003,We also offer bog trips in ARGO amphibious vehicles.,Есть также возможность добраться до болотной деревни и на автомобиле-амфибии ARGO.
950004,Lübeck’s Main Station is a direct bus ride or a 15-minute walk away.,До главного железнодорожного вокзала Любека можно добраться от отеля на прямом автобусе или дойти за 15 минут.
...,...,...
999995,Such actions are called interventions.,Такие действия называются интервенциями.
999996,"Kaliningrad, May 20, 2013","Калининград, 20 мая 2013"
999997,"Urinary tract infections in pregnancy A.I. Dyadyk, A.E. Bagriy, N.F. Yarovaya, M.V. Homenko, D.B. Reznikov, Yu.V. Roschin, I.G. Bagriy Urinary tract infections (UTI) associated with pregnancy represents the important problem of obstetrics, urology and nephrology, caused by changes in clinical features UTI, approaches for assessment and treatment UTI and high risk of obstetrical, urologic and n...","Сообщение 1 А.И. Дядык, А.Э. Багрий, Н.Ф. Яровая, М.В. Хоменко, Д.Б. Резников, Ю.В. Рощин, И.Г. Багрий В первом сообщении приведены данные относительно эпидемиологии и этиологии инфекций мочевыводящих путей (МВП) у беременных, кратко изложены вопросы физиологических изменений МВП в период беременности."
999998,"The default is 8 days for Exchange Server 5.5 and 40 days for Exchange 2000 Server, for Exchange Server 2003, for Exchange Server 2007, and for Exchange Server 2010.","Значение по умолчанию составляет 8 дней для сервера Exchange Server 5.5 и 40 дней для серверов Exchange Server 2000, Exchange Server 2003, Exchange Server 2007 и Exchange Server 2010."


## Кодировщик-декодеровщик

In [3]:
en_sents = open('en.csv', encoding='utf-8').read().splitlines()[:1000]
ru_sents = open('ru.csv', encoding='utf-8').read().splitlines()[:1000]

In [4]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt

tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["en.csv"], trainer=trainer_en)

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_ru.train(files=["ru.csv"], trainer=trainer_ru)

In [5]:
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')

In [6]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
tokenizer_en.token_to_id('[PAD]')
PAD_IDX

3

In [7]:
tokenizer_ru.get_vocab_size()

30000

In [8]:
tokenizer_en.get_vocab_size()

30000

In [9]:
max_len_en, max_len_ru = 40, 45

In [10]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, max_len_ru) for t in ru_sents]

In [11]:
len(X_en), len(X_ru)

(1000, 1000)

In [12]:
X_ru[32]

[1,
 4773,
 1340,
 21,
 6218,
 35,
 248,
 1566,
 1646,
 592,
 2532,
 482,
 1228,
 252,
 495,
 290,
 300,
 469,
 22693,
 5890,
 1917,
 23,
 2]

In [13]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
ru_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

# Токенизация и паддинг
X = tokenizer(en_sents, padding=True, return_tensors="pt")
Y = ru_tokenizer(ru_sents, padding=True, return_tensors="pt")

In [14]:
X

{'input_ids': tensor([[  101,  1010,  1014,  ...,     0,     0,     0],
        [  101, 20317,  8889,  ...,     0,     0,     0],
        [  101, 20317,  8889,  ...,     0,     0,     0],
        ...,
        [  101, 20317,  2683,  ...,     0,     0,     0],
        [  101, 20317,  2683,  ...,     0,     0,     0],
        [  101, 20317,  2683,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [15]:
X = nn.utils.rnn.pad_sequence([
    torch.tensor(x, dtype=torch.int64) for x in X['input_ids']
], padding_value=tokenizer_en.get_vocab_size()).T
Y = nn.utils.rnn.pad_sequence([
    torch.tensor(x, dtype=torch.int64) for x in Y['input_ids']
], padding_value=tokenizer_ru.get_vocab_size()).T

C:\Users\1\AppData\Local\Temp\ipykernel_35232\849731593.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(x, dtype=torch.int64) for x in X['input_ids']
C:\Users\1\AppData\Local\Temp\ipykernel_35232\849731593.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(x, dtype=torch.int64) for x in Y['input_ids']


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([990, 134]),
 torch.Size([990, 165]),
 torch.Size([10, 134]),
 torch.Size([10, 165]))

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

# Гиперпараметры
INPUT_DIM = tokenizer.vocab_size  # Входная длина (source sequence)
OUTPUT_DIM = ru_tokenizer.vocab_size  # Выходная длина (target sequence)
EMBEDDING_DIM = 256  # Размерность embeddings
HIDDEN_DIM = 512  # Размер скрытого слоя GRU
BATCH_SIZE = 50  # Размер батча
LEARNING_RATE = 0.001  # Скорость обучения
N_EPOCHS = 2  # Количество эпох
TEACHER_FORCING_RATIO = 0.5  # Коэффициент teacher forcing

# Кодировщик
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)  # input_dim = INPUT_DIM
        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        
    def forward(self, src):
        embedded = self.embedding(src)  # Преобразование индексов в векторы
        outputs, hidden = self.rnn(embedded)  # RNN обработка
        return hidden
# Декодер
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, input, hidden):
        # input = [batch_size], hidden = [1, batch_size, hidden_dim]
        input = input.unsqueeze(1)  # input = [batch_size, 1]
        embedded = self.embedding(input)  # embedded = [batch_size, 1, embedding_dim]
        output, hidden = self.rnn(embedded, hidden)  # output = [batch_size, 1, hidden_dim]
        prediction = self.fc_out(output.squeeze(1))  # prediction = [batch_size, output_dim]
        return prediction, hidden

# Seq2Seq модель
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.size(1)
        batch_size = trg.size(0)
        outputs = torch.zeros(batch_size, trg_len, OUTPUT_DIM).to(trg.device)
        hidden = self.encoder(src)  # Начальное скрытое состояние от кодировщика
        input = trg[:, 0]  # Начинаем с первого токена (SOS)

        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)  # Извлекаем токен с наибольшей вероятностью
            input = trg[:, t] if teacher_force else top1
        return outputs

In [18]:
encoder = Encoder(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM)
decoder = Decoder(OUTPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM)
model = Seq2Seq(encoder, decoder).to('cuda' if torch.cuda.is_available() else 'cpu')

# Оптимизатор и функция потерь
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=0) 

In [19]:
model.train()

for epoch in range(N_EPOCHS):
    epoch_loss = 0
    for i in range(0, X_train.size(0), BATCH_SIZE):
        src = X_train[i:i + BATCH_SIZE].to('cuda' if torch.cuda.is_available() else 'cpu')
        trg = y_train[i:i + BATCH_SIZE].to('cuda' if torch.cuda.is_available() else 'cpu')
        
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio=TEACHER_FORCING_RATIO)
        
        # Убираем первый токен и приводим к [batch_size * trg_len, output_dim]
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(f'Лосс с батча: {epoch_loss}')
    print('----------------------')
    print(f'Epoch [{epoch+1}/{N_EPOCHS}], Loss: {epoch_loss:.4f}')
    print('----------------------')

Лосс с батча: 11.691032409667969
Лосс с батча: 23.237709999084473
Лосс с батча: 34.61364936828613
Лосс с батча: 45.857863426208496
Лосс с батча: 56.74956130981445
Лосс с батча: 67.17886066436768
Лосс с батча: 77.13906955718994
Лосс с батча: 86.597731590271
Лосс с батча: 95.68481922149658
Лосс с батча: 104.46657276153564
Лосс с батча: 113.0367841720581
Лосс с батча: 121.43892860412598
Лосс с батча: 129.68616008758545
Лосс с батча: 138.14119625091553
Лосс с батча: 146.43154048919678
Лосс с батча: 154.65403079986572
Лосс с батча: 163.12715530395508
Лосс с батча: 171.46352672576904
Лосс с батча: 179.9744644165039
Лосс с батча: 188.27903652191162
----------------------
Epoch [1/2], Loss: 188.2790
----------------------
Лосс с батча: 7.580845355987549
Лосс с батча: 14.971240043640137
Лосс с батча: 22.20725965499878
Лосс с батча: 29.53685474395752
Лосс с батча: 36.65718746185303
Лосс с батча: 43.67736530303955
Лосс с батча: 50.59672689437866
Лосс с батча: 57.5425238609314
Лосс с батча: 64.623

In [26]:
optimizer.zero_grad()
output = model(X_test, y_test, teacher_forcing_ratio=TEACHER_FORCING_RATIO)

In [ ]:
output_dim = output.shape[-1]
output = output[:, 1:].reshape(-1, output_dim)
trg = y_test[:, 1:].reshape(-1)

In [43]:
res = output.softmax(2).argmax(2)
res

tensor([[    0, 30509,   137,  ...,   102,   132,   108],
        [    0, 30509,   137,  ...,   102,   132,   108],
        [    0, 30509,   137,  ...,   102,   132,   108],
        ...,
        [    0, 30509,   137,  ...,   102,   132,   108],
        [    0, 30509,   137,  ...,   102,   132,   108],
        [    0, 30509,   137,  ...,   102,   132,   108]])

In [ ]:
max_indices

In [35]:
decoded_en_sents = [ru_tokenizer.decode(ids, skip_special_tokens=True) for ids in res[:10]]

In [39]:
decoded_ru_sents = [ru_tokenizer.decode(ids, skip_special_tokens=True) for ids in trg]

In [46]:
output

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-5.1773, -5.0126, -4.8821,  ..., -4.6691, -3.8730, -4.7070],
         [-7.9525, -7.2474, -6.6537,  ..., -6.9330, -5.9149, -6.7827],
         ...,
         [-8.2891, -8.7641, -7.9166,  ..., -7.7674, -7.0388, -7.2581],
         [-7.1868, -7.9089, -7.1350,  ..., -6.8353, -6.4027, -6.7858],
         [-7.2471, -8.0015, -7.2900,  ..., -6.9146, -6.9417, -6.5111]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-5.1774, -5.0124, -4.8819,  ..., -4.6693, -3.8730, -4.7070],
         [-7.9525, -7.2474, -6.6537,  ..., -6.9330, -5.9149, -6.7826],
         ...,
         [-8.2901, -8.7646, -7.9160,  ..., -7.7676, -7.0377, -7.2573],
         [-7.1879, -7.9094, -7.1344,  ..., -6.8357, -6.4016, -6.7850],
         [-7.2484, -8.0021, -7.2896,  ..., -6.9150, -6.9408, -6.5108]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-5.1773, -5.0125, -4.8820,  ..., -4

In [40]:
print(decoded_en_sents)

['9500,,,,,,,,,,,.,,,,,,,,,.,,,,,,. ",.,. ",,.,,,.,.,. ",.,,.,.,,. ",,,.,,,,,.,,,,. ",. ",,. ",,. ".,. ".,. ",. ",,,. ",. ".,.,,,,,,. ",. ",.. ",,,. ",.,.,,. ".,,. ". ". ". ". ". "', '9500,,,,,,,,,,,,,,,,,,,,,.,,,.,,,. ". ". ",,.,,,.,,. ",.,,.,.,,. ",,,.,,,,,.,,,,. ",. ",,. ",,. ".,. ".,. ",. ",,,. ",. ".,.,,,,,,. ",. ". ". ",,,. ",.. ",,. ".,,. ". ". ". ". ". "', '9500,,,,,,,,,,,,,,,,,,,,,,,,,,,,. ",.,. ",,.,,,.,.,. ",.,,.,.,,. ",,,.,,,,,.,,,,. ",. ",,. ",,. ".,. ".,. ",. ",,,. ",. ".,.,,,,,,. ",. ",.. ",,,. ",.,.,,. ".,,. ". ". ". ". ". "', '9500,,,,,,,,,,,,,,,,,,,,,.,,,.,,. ",.,. ".,..,,.,. ",.,,.,.,,. ",,,.,,,,,.,,,,. ",. ",,. ",,. ".,. ".,. ",. ",,,. ",. ".,.,,,,,,. ",. ". ". ",,,. ",.. ",,. ".,,. ". ". ". ". ". "', '9500,,,,,,,,,,,,,,,,,,,,,,,,. ",,. ". ". ",,..,,.,.. ",,. ",,.,,. ",,,.,,,,,.,,,,. ",. ",,. ",,. ".,. ".,. ",. ",,,. ",. ".,.,,,,,,. ",. ". ". ",,,. ",.. ",,. ".,,. ". ". ". ". ". "', '9500,,,,,,,,,,,,,,,,,,,,,.,,,,,,. ",,,,.,,.,.,.,.,. ",..,.,.,,. ",,,.,,,,,.,,,,. ",

In [41]:
print(decoded_ru_sents)

['950', '##59', '##5', ',', '"', 'Так', 'же', ',', 'как', 'флаг', ',', 'герб', 'и', 'гимн', '.', '"', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '950', '##39', '##1', ',', '"', 'На', 'сцене', 'менеджер', 'продукта', 'проводил', 'презентацию', ',', 'говоря', 'обычную', 'чеп', '##уху', 'со', 'словами', '«', 'потряс', '##ающий', '»', 'и', '«', 'фантастический', '»', 'и', 'другими', 'хвалеб', '##ными', 'словес', '##ами', ',', 'которыми', 'так', 'любят

In [45]:
ru_tokenizer.decode(137, skip_special_tokens=True)

'##0'

## бибки

In [17]:
training_set = Dataset(X_train, y_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True)

In [18]:
class Encoder(nn.Module):
    def __init__(
        self,
        input_dim: int,
        embedding_dim: int,
        hidden_size_in: int,
        hidden_size_out: int,
        dropout: float,
        rnn_num_layers: int = 1,
        bidirectional: bool = False,
    ) -> None:
        super(Encoder, self).__init__()

        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.hidden_size_in = hidden_size_in
        self.hidden_size_out = hidden_size_out
        self.dropout = dropout
        self.n_layers = rnn_num_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(
            num_embeddings=input_dim,  # vocab_size
            embedding_dim=embedding_dim,
        )

        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_size_in,
            num_layers=rnn_num_layers,
            dropout=dropout,
            bidirectional=bidirectional,
            batch_first=True,
        )
        
        self.fc = nn.Linear(2 * hidden_size_in, hidden_size_out)

    def forward(self, inputs):
        """
            inputs ~ [batch_size, padded_input_doc_len_in_batch]

            output ~ Size([batch_size, padded_input_doc_len_in_batch, bi * hidden_size_in])
            h_n ~ Size([bi * rnn_num_layers, batch_size, hidden_size_in])
            h_cat ~ Size([batch_size, bi * hidden_size_in])
            
            output ~ Size([batch_size, padded_input_doc_len_in_batch, hidden_size_out])
            hidden ~ Size([batch_size, hidden_size_out])

            encoder h_n -> decoder h_0
        """
        embedded = self.embedding(inputs)  #.flip(1)  # [batch_size, padded_input_doc_len_in_batch, embedding_dim]
        output, h_n = self.gru(embedded)
        h_cat = torch.cat([h_n[0, :, :], h_n[1, :, :]], dim=1)  # если bi=1 и rnn_num_layers != 1 ?
        hidden = torch.tanh(self.fc(h_cat))
        output = torch.tanh(self.fc(output))
        return output, hidden
h_n = torch.ones(2, 20, 32)
torch.cat([h_n[0, :, :], h_n[1, :, :]], dim=1).shape
class Attention(nn.Module):
    def __init__(
        self,
        hidden_size_e: int,
        hidden_size_d: int,
    ):
        super(Attention, self).__init__()
        self.hidden_size_e = hidden_size_e
        self.hidden_size_d = hidden_size_d
        self.fc1 = nn.Linear(2 * hidden_size_d, hidden_size_d)
        self.fc2 = nn.Linear(hidden_size_d, 1, bias=False)
    
    def forward(
        self,
        outputs_e: torch.Tensor,
        hidden_d: torch.Tensor,
    ):
        """
            -outputs_e ~ Size([batch_size, input_doc_len, bi * hidden_size_e])
            +outputs_e ~ Size([batch_size, input_doc_len, hidden_size_d])
            hidden_d ~ Size([batch_size, hidden_size_d])
            
            return: Size([batch_size, doc_len])
        """
        # Реализация со скалярными произведениями
        # Если че тут бахнуть слой с активацией 🤯
        hidden = hidden_d.unsqueeze(1).repeat(1, outputs_e.shape[1], 1)
        cat = torch.cat([hidden, outputs_e], dim=2)
        attention = torch.tanh(self.fc1(cat))
        attention = self.fc2(attention).squeeze(dim=2)        
        distributions = torch.softmax(attention, dim=1)
        
#         hidden_d = hidden_d.unsqueeze(2)  # [batch_size, hidden_size_d, 1]
#         scores = torch.bmm(outputs_e, hidden_d)  # [batch_size, input_doc_len, 1]
#         scores = scores.squeeze(2)  # [batch_size, input_doc_len]
#         distributions = scores.softmax(1)  # [batch_size, input_doc_len]
        return distributions
a = torch.ones(20, 7, 8)
b = torch.ones(20, 8)
bb = b.unsqueeze(1).repeat(1, 7, 1)
torch.cat([bb, a], dim=2).shape
# outputs_e ~ Size([batch_size, input_doc_len, bi * hidden_size_e])
# hidden_d ~ Size([1, batch_size, hidden_size_d])
a = torch.ones(20, 7, 8)
b = torch.ones(1, 20, 8)
bb = b.permute(1, 2, 0)  # [20, 8, 1]
torch.bmm(a, bb).squeeze(2).softmax(1)  # [20, 8]

# bb = b.permute(1, 0, 2).repeat(1, 7, 1)  # [20, 7, 8]
# torch.cat([bb, a], dim=2).shape  # [20, 7, 2*8 + 8]
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim: int,
        embedding_dim: int,
        hidden_size_e: int,
        hidden_size: int,
        attention: Attention,
        dropout: float,
        rnn_num_layers: int = 1,
    ) -> None:
        super(Decoder, self).__init__()

        self.attention = attention
        self.output_dim = output_dim
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.n_layers = rnn_num_layers

        self.embedding = nn.Embedding(
            num_embeddings=output_dim,  # макс кол-во векторов = vocab_size
            embedding_dim=embedding_dim,  # заданной размерности
        )

        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=rnn_num_layers,
            dropout=dropout,
            batch_first=True,
        )

        self.fc = nn.Linear(hidden_size, output_dim)


    def forward(
        self,
        input: torch.Tensor,
        hidden: torch.Tensor,
        encoder_outputs: torch.Tensor,
    ):
        """
            input: batch of sos ~ [batch_size]
            hidden ~ [batch_size, hidden_size_d]
            encoder_outputs ~ [batch_size, input_doc_len, hidden_size_d]
        """
        attention_distributes = self.attention(
            outputs_e=encoder_outputs,
            hidden_d=hidden,
        )  # [batch_size, input_doc_len]
        context = torch.bmm(
            attention_distributes.unsqueeze(1),  # [batch_size, 1, input_doc_len]
            encoder_outputs,  # [batch_size, input_doc_len, hidden_size_d]
        )  # [batch_size, 1, hidden_size_d]
        context = context.permute(1, 0, 2)  # [1, batch_size, hidden_size_d]

        inputs = input.unsqueeze(1)  # [batch_size, 1]
        embedded = self.embedding(inputs)  # [batch_size, 1, embedding_dim]

        # output ~ [batch_size, 1, hidden_size_d]
        # h_n ~ [1 * rnn_num_layers, batch_size, hidden_size_d]
        output, h_n = self.gru(embedded, context)

        prediction = self.fc(h_n.squeeze(0))  # [batch_size, output_size]
        h_n = h_n.squeeze(0)  # [batch_size, hidden_size_d]
        return prediction, h_n
class Seq2Seq(nn.Module):
    def __init__(
        self,
        encoder: Encoder,
        decoder: Decoder,
        device,
    ) -> None:
        super(Seq2Seq, self).__init__()

        self.device = device
        self.encoder = encoder
        self.decoder = decoder

#         assert encoder.hidden_size == decoder.hidden_size, 'hidden_size of Encoder and Decoder did not match'
        assert encoder.n_layers == decoder.n_layers, 'n_layers of Encoder and Decoder did not match'

    def forward(
        self,
        inputs,
        target_outputs,
        teacher_forcing_ratio: float = 0.,
    ):
        """
            inputs ~ [batch_size, padded_in_batch_input_doc_len]
            target_outputs ~ [batch_size, padded_in_batch_target_doc_len]
        """
        batch_size, max_len = target_outputs.shape
        target_vocab_size = self.decoder.output_dim

        target_pad = 0
        target_sos = 1
        
        # outputs ~ [max_len, batch_size, target_vocab_size]
        outputs = torch.empty(max_len, batch_size, target_vocab_size, device=self.device).fill_(target_sos)

        # encoder_outputs ~ [batch_size, input_doc_len, hidden_size_d]
        # hidden ~ [batch_size, hidden_size_d]
        encoder_outputs, hidden = self.encoder(inputs)

#         input = torch.empty(batch_size, device=self.device).fill_(target_sos).type(torch.int64)
        input = target_outputs[:, 0]  # batch of sos ~ [batch_size]
        outputs[0, :, target_sos] = 1

        for t in range(1, max_len):
            # pred ~ [batch_size, target_vocab_size]
            # hidden ~ [1 * rnn_num_layers, batch_size, hidden_size_d]
            pred, hidden = self.decoder(
                input=input,  # [batch_size]
                hidden=hidden,  # [batch_size, hidden_size_d]
                encoder_outputs=encoder_outputs,  # [batch_size, input_doc_len, hidden_size_d]
            )

            outputs[t] = pred

            if np.random.sample() < teacher_forcing_ratio:
                input = target_outputs[:, t]  # Size([batch_size])
            else:
                # Q: 2x softmax?
                # Q: можно ли не брать софтмакс?
                input = pred.softmax(1).argmax(1)  # Size([batch_size])

        return outputs.permute(1, 0, 2)  # [batch_size, max_len, target_vocab_size]
a = torch.zeros(3, 2, 5)
a[0, :, 1] = 1
a.permute(1, 0, 2).softmax(2).argmax(2)
(
    torch.tensor([-1, -2, -2, -3.]).softmax(0),
    torch.tensor([1, 2, 2, 3.]).softmax(0),
)

(tensor([0.5344, 0.1966, 0.1966, 0.0723]),
 tensor([0.0723, 0.1966, 0.1966, 0.5344]))

In [19]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [20]:
lr = 1e-3
loss = nn.CrossEntropyLoss(ignore_index=0)

# RU to EN

embedding_dim = 30000
hidden_size = 512
rnn_num_layers = 1
encoder = Encoder(
    input_dim=tokenizer_en.get_vocab_size(),
    embedding_dim=embedding_dim,
    hidden_size_in=hidden_size,
    hidden_size_out=hidden_size,
    dropout=0.5,
    rnn_num_layers=rnn_num_layers,
    bidirectional=True,
)
attention = Attention(
    hidden_size_e=hidden_size,
    hidden_size_d=hidden_size,
)
decoder = Decoder(
    output_dim=tokenizer_ru.get_vocab_size(),
    embedding_dim=embedding_dim,
    hidden_size_e=hidden_size,
    hidden_size=hidden_size,
    attention=attention,
    dropout=0.5,
    rnn_num_layers=rnn_num_layers,
)
seq2seq = Seq2Seq(
    encoder=encoder,
    decoder=decoder,
    device=DEVICE,
)
seq2seq = nn.DataParallel(seq2seq)
seq2seq.to(DEVICE)
optimizer = torch.optim.Adam(seq2seq.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

all(all(param.is_cuda for param in model.parameters()) for model in (encoder, decoder, seq2seq))

C:\Users\1\Jupyter Notebook\venv\lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


False

In [21]:
seq2seq

DataParallel(
  (module): Seq2Seq(
    (encoder): Encoder(
      (embedding): Embedding(30000, 30000)
      (gru): GRU(30000, 512, batch_first=True, dropout=0.5, bidirectional=True)
      (fc): Linear(in_features=1024, out_features=512, bias=True)
    )
    (decoder): Decoder(
      (attention): Attention(
        (fc1): Linear(in_features=1024, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=1, bias=False)
      )
      (embedding): Embedding(30000, 30000)
      (gru): GRU(30000, 512, batch_first=True, dropout=0.5)
      (fc): Linear(in_features=512, out_features=30000, bias=True)
    )
  )
)

In [22]:
epochs = 1

for epoch in range(epochs):
    print(f'[Seq2Seq] Epoch {epoch + 1}...')
    optimizer.zero_grad()
    outputs = seq2seq(X_train, y_train)
    loss_value = loss(outputs.permute(0, 2, 1), y_train)
    print(f'Ошибка: {loss_value}')
    loss_value.backward()
    torch.nn.utils.clip_grad_norm_(seq2seq.parameters(), 1)
    optimizer.step()
    scheduler.step()

[Seq2Seq] Epoch 1...


IndexError: index out of range in self

In [12]:
max_value = max(max(sub_array) for sub_array in X_ru)

print(f"Максимальное значение: {max_value}")

Максимальное значение: 29997


In [13]:
max_value = max(max(sub_array) for sub_array in X_en)

print(f"Максимальное значение: {max_value}")

Максимальное значение: 29990
